In [2]:
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
import json
import time
from datetime import date, datetime, timedelta
from dateutil.easter import easter

import matplotlib as plt
import seaborn as sns

#! pip install yfinance
import yfinance as yf

#!pip install akshare
import akshare as ak

#!pip install pandas pandas-datareader
from pandas_datareader.data import DataReader
from pandas_datareader import wb



In [3]:
# skip cells:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):

    return

# Data

## Data Exploration

In [4]:
df_walmart_train = pd.read_csv('csv_files/walmart_data/train.csv')
df_walmart_test = pd.read_csv('csv_files/walmart_data/test.csv')
df_walmart_features = pd.read_csv('csv_files/walmart_data/features.csv')
df_walmart_stores = pd.read_csv('csv_files/walmart_data/stores.csv')

df_walmart_train["Date"] = pd.to_datetime(df_walmart_train["Date"], errors="raise")
df_walmart_test["Date"] = pd.to_datetime(df_walmart_test["Date"], errors="raise")
df_walmart_features["Date"] = pd.to_datetime(df_walmart_features["Date"], errors="raise")

In [5]:
df_walmart_train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [6]:
df_walmart_test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


In [7]:
df_walmart_features


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


## Data Gathering

#### Sp500

In [8]:
%%skip
# SP500 Index
START_DATE = "2000-01-01"
END_DATE   = date.today().isoformat()


daily = yf.download(
    "^GSPC",           # S&P 500 index ticker
    start=START_DATE,
    end=END_DATE,
    interval="1d",
    auto_adjust=True,
    progress=False,
)

if isinstance(daily.columns, pd.MultiIndex):
    lvl0 = daily.columns.get_level_values(0)
    if "Close" in lvl0:
        close_prices = daily.xs("Close", axis=1, level=0)
    elif "Adj Close" in lvl0:
        close_prices = daily.xs("Adj Close", axis=1, level=0)
    else:
        raise KeyError("Neither 'Close' nor 'Adj Close' found in data")
else:
    if "Close" in daily.columns:
        close_prices = daily["Close"]
    else:
        close_prices = daily["Adj Close"]


weekly_mean_close = close_prices.resample("W-FRI").mean()

if isinstance(weekly_mean_close, pd.Series):
    df_sp500 = weekly_mean_close.to_frame(name="SPX_Weekly_Mean_Close")
else:
    df_sp500 = weekly_mean_close.copy()
    df_sp500.columns = ["SPX_Weekly_Mean_Close"]


OUTFILE = "csv_files/idea_csv/sp500_weekly_mean_close.csv"
df_sp500.to_csv(OUTFILE)



In [9]:
df_sp500 = pd.read_csv("csv_files/idea_csv/sp500_weekly_mean_close.csv")
df_sp500

,Date,SPX_Weekly_Mean_Close
0,2000-01-07,1420.333984
1,2000-01-14,1448.648022
2,2000-01-21,1449.492493
3,2000-01-28,1394.874023
4,2000-02-04,1412.439990
...,...,...
1324,2025-05-23,5878.699902
1325,2025-05-30,5908.487427
1326,2025-06-06,5963.355957
1327,2025-06-13,6017.832031


#### Walmart Stock Price

In [10]:
%%skip
START_DATE = "2000-01-01"
END_DATE   = date.today().isoformat()


daily = yf.download(
    "WMT",           
    start=START_DATE,
    end=END_DATE,
    interval="1d",
    auto_adjust=True, 
    progress=False,
)


if isinstance(daily.columns, pd.MultiIndex):
    lvl0 = daily.columns.get_level_values(0)
    if "Close" in lvl0:
        close_prices = daily.xs("Close", axis=1, level=0)
    elif "Adj Close" in lvl0:
        close_prices = daily.xs("Adj Close", axis=1, level=0)
    else:
        raise KeyError("Neither 'Close' nor 'Adj Close' found in data")
else:
    if "Close" in daily.columns:
        close_prices = daily["Close"]
    else:
        close_prices = daily["Adj Close"]


weekly_mean_close = close_prices.resample("W-FRI").mean()


if isinstance(weekly_mean_close, pd.Series):
    df_walmart_stock = weekly_mean_close.to_frame(name="WMT_Weekly_Mean_Close")
else:
    df_walmart_stock = weekly_mean_close.copy()
    df_walmart_stock.columns = ["WMT_Weekly_Mean_Close"]


OUTFILE = "csv_files/idea_csv/wmt_weekly_mean_close.csv"
df_walmart_stock.to_csv(OUTFILE)



In [11]:
df_walmart_stock = pd.read_csv("csv_files/idea_csv/wmt_weekly_mean_close.csv")
df_walmart_stock

,Date,WMT_Weekly_Mean_Close
0,2000-01-07,13.941764
1,2000-01-14,14.021870
2,2000-01-21,13.642020
3,2000-01-28,12.676027
4,2000-02-04,12.240765
...,...,...
1324,2025-05-23,96.924001
1325,2025-05-30,97.660000
1326,2025-06-06,98.906000
1327,2025-06-13,95.968001


#### External Logistic companies Walmart

In [12]:
%%skip
#   - ARCB: ArcBest Corporation (ABF Logistics / ArcBest Freight)
#   - AIT: AIT Worldwide Logistics
#   - CEVA: CEVA Logistics
#   - DPW.DE: Deutsche Post (DHL Freight / DHL Supply Chain) on XETRA
#   - FDX: FedEx Corporation (FedEx Freight)
#   - SAIA: Saia, Inc. (Saia Motor Freight Line)
#   - TFII.TO: TFI International (TForce Freight) on TSX
#   - XPO: XPO Logistics, Inc.
#   - ODFL: Old Dominion Freight Line, Inc.
#   - UPS: United Parcel Service, Inc.
#   - JBHT: J.B. Hunt Transport Services, Inc.
# Note: Some private carriers (Estes Express, R+L Carriers) are not publicly traded.

TICKERS = [
    "ARCB", "AIT", "CEVA", "DPW.DE", "FDX",
    "SAIA", "TFII.TO", "XPO", "ODFL", "UPS", "JBHT"
]

START_DATE = "2000-01-01"
END_DATE   = date.today().isoformat()

daily = yf.download(
    TICKERS,
    start=START_DATE,
    end=END_DATE,
    interval="1d",
    auto_adjust=True,
    group_by="ticker",
    threads=True,
    progress=True,
)

close = pd.DataFrame()
for sym in TICKERS:
    try:
        series = daily[sym]["Close"]
        close[sym] = series
    except Exception:
        print(f"Skipping {sym!r}: no data available or ticker invalid")


before = close.shape[1]
close = close.dropna(axis=1, how="all")
after = close.shape[1]
print(f"Dropped {before-after} tickers; {after} tickers remain for analysis")

df_logistics = close.resample("W-FRI").mean().round(4)

df_logistics.columns = [f"{sym}_df_logistics_Close" for sym in df_logistics.columns]

OUTFILE = "csv_files/idea_csv/logistics_df_logistics_close.csv"
df_logistics.to_csv(OUTFILE)


In [13]:
df_logistics = pd.read_csv("csv_files/idea_csv/logistics_df_logistics_close.csv")
df_logistics

,Date,ARCB_df_logistics_Close,AIT_df_logistics_Close,CEVA_df_logistics_Close,FDX_df_logistics_Close,SAIA_df_logistics_Close,TFII.TO_df_logistics_Close,XPO_df_logistics_Close,ODFL_df_logistics_Close,UPS_df_logistics_Close,JBHT_df_logistics_Close
0,2000-01-07,10.2456,4.2252,NaN,36.1330,NaN,NaN,NaN,0.4840,33.4329,2.6094
1,2000-01-14,10.2209,4.0500,NaN,37.6513,NaN,NaN,NaN,0.4914,33.7016,2.5821
2,2000-01-21,10.6344,4.3393,NaN,35.1930,NaN,NaN,NaN,0.5243,34.1281,2.5124
3,2000-01-28,9.9543,4.2880,NaN,33.1066,NaN,NaN,NaN,0.5354,32.7017,2.4714
4,2000-02-04,9.3568,4.2649,NaN,31.9144,NaN,NaN,NaN,0.5190,29.3584,2.5079
...,...,...,...,...,...,...,...,...,...,...,...
1324,2025-05-23,63.7080,226.5780,20.1420,219.8980,272.0360,121.3875,121.308,165.8267,96.9700,141.4160
1325,2025-05-30,63.9625,227.6050,19.1925,218.8175,269.3875,119.2300,117.125,161.9264,97.4825,139.3725
1326,2025-06-06,63.4820,228.9500,19.8700,218.5800,260.4360,118.7120,115.588,160.7950,97.8220,139.3440
1327,2025-06-13,69.1700,230.5940,21.0920,224.7040,259.1800,123.1960,121.270,162.9600,100.7700,141.6980


#### Official China PMI (Caixin PMI only starts in 2014)


In [14]:
%%skip
# All AkShare functions containing "pmi"

df_official = ak.macro_china_pmi()

print("Columns in df_official:", df_official.columns.tolist())


In [15]:
%%skip
# 1) Fetch the official China PMI data
df_official = ak.macro_china_pmi()

# 2) Rename Chinese column names to English
df_official = df_official.rename(columns={
    "月份": "Date",
    "制造业-指数": "Official_Manufacturing_PMI",
    "制造业-同比增长": "Official_Manufacturing_PMI_YoY",
    "非制造业-指数": "Official_Services_PMI",
    "非制造业-同比增长": "Official_Services_PMI_YoY",
})

# 3) Clean and parse the 'Date' column ("YYYY年MM月份" → "YYYY-MM")
df_official["Date"] = (
    df_official["Date"]
      .str.replace("年", "-", regex=False)
      .str.replace("月份", "", regex=False)
)
df_official["Date"] = pd.to_datetime(df_official["Date"], format="%Y-%m")

# 4) Set Date as the index and sort
df_official = df_official.set_index("Date").sort_index()

# 5) Subset to the period 2009-01-01 through 2014-12-31
df_pmi_china = df_official.loc["2009-01-01":"2014-12-31"]
df_pmi_china.to_csv("csv_files/idea_csv/df_pmi_china.csv")

In [16]:
df_pmi_china = pd.read_csv("csv_files/idea_csv/df_pmi_china.csv")
df_pmi_china

,Date,Official_Manufacturing_PMI,Official_Manufacturing_PMI_YoY,Official_Services_PMI,Official_Services_PMI_YoY
0,2009-01-01,45.3,-14.528302,53.7,-10.797342
1,2009-02-01,49.0,-8.239700,55.1,-7.082631
2,2009-03-01,52.4,-10.273973,54.4,-7.640068
3,2009-04-01,53.5,-9.628378,53.5,-8.390411
4,2009-05-01,53.1,-0.375235,54.9,-4.355401
...,...,...,...,...,...
67,2014-08-01,51.1,0.196078,54.4,0.927644
68,2014-09-01,51.1,0.000000,54.0,-2.527076
69,2014-10-01,50.8,-1.167315,53.8,-4.440497
70,2014-11-01,50.3,-2.140078,53.9,-3.750000


#### PCE USA (Personal Consumption Expenditures)


In [17]:
%%skip
START_DATE = "2000-01-01"
END_DATE   = date.today().isoformat()

df_pce = DataReader("PCE", "fred", start=START_DATE, end=END_DATE)

df_pce.columns = ["Personal_Consumption_Expenditures"]

# 4) Save and quick sanity-check
OUTFILE = "csv_files/idea_csv/personal_consumption_expenditures.csv"
df_pce.to_csv(OUTFILE)


In [18]:
df_pce = pd.read_csv("csv_files/idea_csv/personal_consumption_expenditures.csv")
df_pce

,DATE,Personal_Consumption_Expenditures
0,2000-01-01,6542.9
1,2000-02-01,6625.3
2,2000-03-01,6686.5
3,2000-04-01,6679.1
4,2000-05-01,6709.7
...,...,...
299,2024-12-01,20408.1
300,2025-01-01,20389.0
301,2025-02-01,20469.3
302,2025-03-01,20621.8


#### Interest Rates USA (Fed Funds Rate & Tbill 3 Months Yield)

In [19]:
%%skip
START = "2001-06-01"               
END   = date.today().isoformat()


fed  = DataReader("FEDFUNDS", "fred", START, END)
tbill = DataReader("DGS3MO",  "fred", START, END)

df_interest_rates = pd.concat([fed, tbill], axis=1).rename(columns={
    "FEDFUNDS": "Fed_Funds_Rate",
    "DGS3MO":   "TBill_3mo_Yield",
})
df_interest_rates = df_interest_rates.resample("W-FRI").mean()

OUTFILE = "csv_files/idea_csv/df_interest_rates.csv"
df_interest_rates.to_csv(OUTFILE)

In [20]:
df_interest_rates = pd.read_csv("csv_files/idea_csv/df_interest_rates.csv")
df_interest_rates

,DATE,Fed_Funds_Rate,TBill_3mo_Yield
0,2001-06-01,3.97,3.670
1,2001-06-08,NaN,3.640
2,2001-06-15,NaN,3.558
3,2001-06-22,NaN,3.496
4,2001-06-29,NaN,3.546
...,...,...,...
1250,2025-05-16,NaN,4.398
1251,2025-05-23,NaN,4.372
1252,2025-05-30,NaN,4.355
1253,2025-06-06,NaN,4.436


#### CCI USA (Consumer Confidence Index) from University of Michigan

In [21]:
%%skip
START_DATE = "2001-06-17"
END_DATE   = date.today().isoformat()

df_us_cci = DataReader("UMCSENT", "fred", START_DATE, END_DATE)
df_us_cci.columns = ["Consumer_Sentiment_UMich"]

OUTFILE = "csv_files/idea_csv/consumer_confidence_index.csv"
df_us_cci.to_csv(OUTFILE)

In [22]:
df_us_cci = pd.read_csv("csv_files/idea_csv/consumer_confidence_index.csv")
df_us_cci

,DATE,Consumer_Sentiment_UMich
0,2001-07-01,92.4
1,2001-08-01,91.5
2,2001-09-01,81.8
3,2001-10-01,82.7
4,2001-11-01,83.9
...,...,...
281,2024-12-01,74.0
282,2025-01-01,71.7
283,2025-02-01,64.7
284,2025-03-01,57.0


#### U.S.A Advance Retail Sales: Retail Trade and Food Services

In [23]:
%%skip
START_DATE = "2000-01-01"
END_DATE   = date.today().isoformat()

# RSAFS = Advance Retail Sales: Retail Trade and Food Services (Millions of Dollars, SA)
df_us_retail = DataReader("RSAFS", "fred", START_DATE, END_DATE)

df_us_retail.columns = ["Retail_Sales_Retail_and_Food_Services_USA"]

OUTFILE = "csv_files/idea_csv/usa_retail_sales.csv"
df_us_retail.to_csv(OUTFILE)

In [24]:
df_us_retail = pd.read_csv("csv_files/idea_csv/usa_retail_sales.csv")
df_us_retail

,DATE,Retail_Sales_Retail_and_Food_Services_USA
0,2000-01-01,261545
1,2000-02-01,265686
2,2000-03-01,269019
3,2000-04-01,264067
4,2000-05-01,265992
...,...,...
300,2025-01-01,711461
301,2025-02-01,711757
302,2025-03-01,722572
303,2025-04-01,721983


#### Exchange Rates (China, Mexico, Canada, India, Vietnam)


In [25]:
%%skip
START = "2001-06-17"                 
END   = date.today().isoformat()

#    China (CNY per USD): DEXCHUS  
#    Mexico (MXN per USD): DEXMXUS  
#    Canada (CAD per USD): DEXCAUS  
#    India (INR per USD): DEXINUS  
cny = DataReader("DEXCHUS", "fred", START, END)
mxn = DataReader("DEXMXUS", "fred", START, END)
cad = DataReader("DEXCAUS", "fred", START, END)
inr = DataReader("DEXINUS", "fred", START, END)

# Vietnam via yfinance
vn_df = yf.download(
    "USDVND=X",
    start=START,
    end=END,
    progress=False
)
vn = vn_df[["Close"]].rename(columns={"Close": "VND_per_USD"})


fx = pd.concat([cny, mxn, cad, inr, vn], axis=1).rename(columns={
    "DEXCHUS": "CNY_per_USD",
    "DEXMXUS": "MXN_per_USD",
    "DEXCAUS": "CAD_per_USD",
    "DEXINUS": "INR_per_USD"
})

df_fx = fx.resample("W-FRI").mean().dropna(how="all").round(4)


OUTFILE = "csv_files/idea_csv/foreign_exchange.csv"
df_fx.to_csv(OUTFILE)

In [26]:
df_fx = pd.read_csv("csv_files/idea_csv/foreign_exchange.csv")
df_fx

,Unnamed: 0,CNY_per_USD,MXN_per_USD,CAD_per_USD,INR_per_USD,"('VND_per_USD', 'USDVND=X')"
0,2001-06-22,8.2771,9.0712,1.5308,47.028,NaN
1,2001-06-29,8.2768,9.0695,1.5184,47.064,NaN
2,2001-07-06,8.2768,9.0672,1.5146,47.175,NaN
3,2001-07-13,8.2768,9.2520,1.5270,47.200,NaN
4,2001-07-20,8.2768,9.1962,1.5404,47.162,NaN
...,...,...,...,...,...,...
1248,2025-05-23,7.2038,19.3119,1.3864,85.558,25944.4
1249,2025-05-30,7.1934,19.3442,1.3788,85.420,25938.4
1250,2025-06-06,7.1865,19.2013,1.3688,85.646,26041.2
1251,2025-06-13,7.1827,18.9487,1.3642,85.670,26018.4


#### US External Tax Rate

In [27]:
%%skip
INDICATOR   = "TM.TAX.MRCH.WM.AR.ZS"  # Tariff rate %
COUNTRIES   = ["CN", "IN", "MX", "CA", "VN"]
START_YEAR  = 2000
END_YEAR    = date.today().year

# from World Bank
df_us_tariff = wb.download(
    indicator=INDICATOR,
    country=COUNTRIES,
    start=START_YEAR,
    end=END_YEAR
)

df_us_tariff = df_us_tariff.reset_index().pivot(index="year", columns="country", values=INDICATOR)

df_us_tariff = df_us_tariff.rename(columns={
    "CN": "China_Applied_Tariff_%", 
    "IN": "India_Applied_Tariff_%", 
    "MX": "Mexico_Applied_Tariff_%", 
    "CA": "Canada_Applied_Tariff_%", 
    "VN": "Vietnam_Applied_Tariff_%"
})


OUTFILE = "csv_files/idea_csv/external_tax_rates.csv"
df_us_tariff.to_csv(OUTFILE, index_label="Year")

In [28]:
df_us_tariff = pd.read_csv("csv_files/idea_csv/external_tax_rates.csv")
df_us_tariff

,Year,Canada,China,India,Mexico,Viet Nam
0,2000,1.31,14.67,23.36,15.15,NaN
1,2001,1.60,14.11,26.51,15.24,15.29
2,2002,1.49,7.72,23.08,5.03,14.73
3,2003,2.46,6.48,21.45,15.44,11.79
4,2004,1.51,5.96,22.96,3.00,11.69
5,2005,1.46,4.87,13.90,3.00,11.35
6,2006,1.45,4.25,8.99,2.39,10.21
7,2007,1.54,5.07,11.99,2.41,9.13
8,2008,1.54,4.47,5.98,1.87,5.03
9,2009,1.70,3.94,6.84,1.61,6.76


##### Holidays

In [29]:
def nth_weekday(year, month, weekday, n):
    """
    Return the date of the nth occurrence of the given weekday
    in the specified month and year.
    weekday: Monday=0, Sunday=6
    """
    d = date(year, month, 1)
    count = 0
    while True:
        if d.weekday() == weekday:
            count += 1
            if count == n:
                return d
        d += timedelta(days=1)

years = range(2010, 2014)
records = []

for year in years:
    # Fixed‐date holidays
    records.append({"Date": pd.Timestamp(date(year, 2, 14)), "Holiday": "Valentine's Day"})
    records.append({"Date": pd.Timestamp(date(year, 7, 4)),  "Holiday": "Independence Day"})
    records.append({"Date": pd.Timestamp(date(year, 10, 31)),"Holiday": "Halloween"})
    records.append({"Date": pd.Timestamp(date(year, 12, 24)),"Holiday": "Christmas Eve"})
    records.append({"Date": pd.Timestamp(date(year, 12, 25)),"Holiday": "Christmas Day"})
    records.append({"Date": pd.Timestamp(date(year, 12, 31)),"Holiday": "New Year's Eve"})
    
    # Mother's Day: 2nd Sunday in May
    md = nth_weekday(year, 5, 6, 2)
    records.append({"Date": pd.Timestamp(md), "Holiday": "Mother's Day"})
    
    # Father's Day: 3rd Sunday in June
    fd = nth_weekday(year, 6, 6, 3)
    records.append({"Date": pd.Timestamp(fd), "Holiday": "Father's Day"})
    
    # Memorial Day: last Monday in May
    d_mem = date(year, 5, 31)
    while d_mem.weekday() != 0:  # 0 = Monday
        d_mem -= timedelta(days=1)
    records.append({"Date": pd.Timestamp(d_mem), "Holiday": "Memorial Day"})
    
    # Labor Day: 1st Monday in September
    ld = nth_weekday(year, 9, 0, 1)
    records.append({"Date": pd.Timestamp(ld), "Holiday": "Labor Day"})
    
    # Easter Sunday
    eas = easter(year)
    records.append({"Date": pd.Timestamp(eas), "Holiday": "Easter Sunday"})
    
    # Thanksgiving: 4th Thursday in November
    th = nth_weekday(year, 11, 3, 4)
    records.append({"Date": pd.Timestamp(th), "Holiday": "Thanksgiving"})
    
    # Black Friday: day after Thanksgiving
    bf = th + timedelta(days=1)
    records.append({"Date": pd.Timestamp(bf), "Holiday": "Black Friday"})
    
    # Small Business Saturday: Saturday after Thanksgiving
    sbs = th + timedelta(days=2)
    records.append({"Date": pd.Timestamp(sbs), "Holiday": "Small Business Saturday"})
    
    # Cyber Monday: Monday after Thanksgiving
    cm = th + timedelta(days=4)
    records.append({"Date": pd.Timestamp(cm), "Holiday": "Cyber Monday"})
    
    # Super Saturday: _last Saturday before_ Christmas Eve
    d2 = date(year, 12, 24) - timedelta(days=1)
    while d2.weekday() != 5:  # 5 = Saturday
        d2 -= timedelta(days=1)
    records.append({"Date": pd.Timestamp(d2), "Holiday": "Super Saturday"})
    
    # Green Monday: 2nd Monday in December
    gm = nth_weekday(year, 12, 0, 2)
    records.append({"Date": pd.Timestamp(gm), "Holiday": "Green Monday"})

# Build and save the DataFrame
df_us_holidays = pd.DataFrame(records).sort_values("Date").reset_index(drop=True)
df_us_holidays.to_csv("csv_files/idea_csv/df_us_holidays.csv", index=False)


In [30]:
df_us_holidays = pd.read_csv("csv_files/idea_csv/df_us_holidays.csv")
print(df_us_holidays)

          Date          Holiday
0   2010-02-14  Valentine's Day
1   2010-04-04    Easter Sunday
2   2010-05-09     Mother's Day
3   2010-05-31     Memorial Day
4   2010-06-20     Father's Day
..         ...              ...
63  2013-12-09     Green Monday
64  2013-12-21   Super Saturday
65  2013-12-24    Christmas Eve
66  2013-12-25    Christmas Day
67  2013-12-31   New Year's Eve

[68 rows x 2 columns]


#### Tax Return

Train

In [31]:
df_tax_return_train = df_walmart_train[["Date"]].copy()
df_tax_return_train


,Date
0,2010-02-05
1,2010-02-12
2,2010-02-19
3,2010-02-26
4,2010-03-05
...,...
421565,2012-09-28
421566,2012-10-05
421567,2012-10-12
421568,2012-10-19


In [32]:

ramp_up_days   = 14 
ramp_down_days = 42  

def filing_deadline(year):
    """
    IRS filing deadline April 15, bumped to Monday if on a weekend.
    """
    d = date(year, 4, 15)
    while d.weekday() >= 5:  # 
        d += timedelta(days=1)
    return pd.Timestamp(d)


def tax_return_weight(ts, ramp_up=ramp_up_days, ramp_down=ramp_down_days):
    """
    Smooth raised-cosine weight:
      • 0 before (deadline - ramp_up)
      • ramps up from 0→1 over `ramp_up` days
      • ramps down from 1→0 over `ramp_down` days
      • 0 after (deadline + ramp_down)
    """
    ts   = pd.Timestamp(ts).normalize()
    peak = filing_deadline(ts.year)
    start = peak - timedelta(days=ramp_up)
    end   = peak + timedelta(days=ramp_down)

    if ts < start or ts > end:
        return 0.0

    if ts <= peak:
        # fraction of ramp-up completed [0…1]
        x = (ts - start).days / ramp_up
        # raised‐cosine from 0→1
        return 0.5 * (1 - np.cos(np.pi * x))
    else:
        # fraction of ramp-down completed [0…1]
        x = (ts - peak).days / ramp_down
        # raised‐cosine from 1→0
        return 0.5 * (1 + np.cos(np.pi * x))


df_tax_return_train = df_walmart_train[['Date']].copy()
df_tax_return_train['TaxReturnImpact'] = (
    df_tax_return_train['Date']
      .dt.normalize()
      .map(tax_return_weight)
)

df_tax_return_train.to_csv('csv_files/idea_csv/df_tax_return_train.csv', index=False)


In [33]:
df_unique = df_tax_return_train.drop_duplicates(subset='Date', keep='first')

In [34]:
df_unique

,Date,TaxReturnImpact
0,2010-02-05,0.0
1,2010-02-12,0.0
2,2010-02-19,0.0
3,2010-02-26,0.0
4,2010-03-05,0.0
...,...,...
138,2012-09-28,0.0
139,2012-10-05,0.0
140,2012-10-12,0.0
141,2012-10-19,0.0


In [35]:
df_tax_return_train

,Date,TaxReturnImpact
0,2010-02-05,0.0
1,2010-02-12,0.0
2,2010-02-19,0.0
3,2010-02-26,0.0
4,2010-03-05,0.0
...,...,...
421565,2012-09-28,0.0
421566,2012-10-05,0.0
421567,2012-10-12,0.0
421568,2012-10-19,0.0


In [36]:
df_tax_return_train = pd.read_csv('csv_files/idea_csv/df_tax_return_train.csv')

Test

In [37]:

ramp_up_days   = 14 
ramp_down_days = 42  

def filing_deadline(year):
    """
    IRS filing deadline April 15, bumped to Monday if on a weekend.
    """
    d = date(year, 4, 15)
    while d.weekday() >= 5:
        d += timedelta(days=1)
    return pd.Timestamp(d)

# ─── SMOOTH WEIGHT FUNCTION ───────────────────────────────
def tax_return_weight(ts, ramp_up=ramp_up_days, ramp_down=ramp_down_days):
    """
    Smooth raised-cosine weight:
      • 0 before (deadline - ramp_up)
      • ramps up from 0→1 over `ramp_up` days
      • ramps down from 1→0 over `ramp_down` days
      • 0 after (deadline + ramp_down)
    """
    ts   = pd.Timestamp(ts).normalize()
    peak = filing_deadline(ts.year)
    start = peak - timedelta(days=ramp_up)
    end   = peak + timedelta(days=ramp_down)

    if ts < start or ts > end:
        return 0.0

    if ts <= peak:
        # fraction of ramp-up completed [0…1]
        x = (ts - start).days / ramp_up
        # raised‐cosine from 0→1
        return 0.5 * (1 - np.cos(np.pi * x))
    else:
        # fraction of ramp-down completed [0…1]
        x = (ts - peak).days / ramp_down
        # raised‐cosine from 1→0
        return 0.5 * (1 + np.cos(np.pi * x))

df_tax_return_test = df_walmart_test[['Date']].copy()
df_tax_return_test['TaxReturnImpact'] = (
    df_tax_return_test['Date']
      .dt.normalize()
      .map(tax_return_weight)
)

df_tax_return_test.to_csv('csv_files/idea_csv/df_tax_return_test.csv', index=False)


In [38]:
df_tax_return_test = pd.read_csv('csv_files/idea_csv/df_tax_return_test.csv')

In [39]:
df_tax_return_test

,Date,TaxReturnImpact
0,2012-11-02,0.0
1,2012-11-09,0.0
2,2012-11-16,0.0
3,2012-11-23,0.0
4,2012-11-30,0.0
...,...,...
115059,2013-06-28,0.0
115060,2013-07-05,0.0
115061,2013-07-12,0.0
115062,2013-07-19,0.0


#### Stores Types & Sizes

In [40]:
df_walmart_train['Store']  = df_walmart_train['Store'].astype(str)
df_walmart_stores['Store'] = df_walmart_stores['Store'].astype(str)

# Merge the store metadata into your training DataFrame
df_store_types_sizes = df_walmart_train.merge(
    df_walmart_stores,     
    on='Store',             
    how='left',       
    validate='many_to_one'  
)

In [41]:
df_store_types_sizes = df_store_types_sizes.loc[:, ["Store","Type","Size"]]

In [42]:
df_store_types_sizes

,Store,Type,Size
0,1,A,151315
1,1,A,151315
2,1,A,151315
3,1,A,151315
4,1,A,151315
...,...,...,...
421565,45,B,118221
421566,45,B,118221
421567,45,B,118221
421568,45,B,118221


#### Oil Price The U.S. domestic

In [43]:
%%skip
start = "2010-02-05"
end   = date.today().isoformat()
 
df_us_oil_price = DataReader("DCOILWTICO", "fred", start, end)

wti_weekly = df_us_oil_price.resample("W-FRI").mean().rename(
    columns={"DCOILWTICO":"WTI_Weekly_Mean_Price"}
)
df_us_oil_price.to_csv('csv_files/idea_csv/df_us_oil_price.csv')

In [44]:
df_us_oil_price = pd.read_csv('csv_files/idea_csv/df_us_oil_price.csv')

In [45]:
df_us_oil_price

,DATE,DCOILWTICO
0,2010-02-05,71.15
1,2010-02-08,71.87
2,2010-02-09,73.71
3,2010-02-10,74.48
4,2010-02-11,75.23
...,...,...
3997,2025-06-03,64.10
3998,2025-06-04,63.57
3999,2025-06-05,64.06
4000,2025-06-06,65.30


#### U.S. ISM Manufacturing PMI & ISM Services PMI


In [46]:
%%skip
# Get columns names
df_man = ak.macro_usa_ism_pmi()

print("Columns in df_man:", df_man.columns.tolist())


In [47]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_man = ak.macro_usa_ism_pmi()

df_man = df_man.rename(columns={
    "日期": "Date",
    "今值": "ISM_Manufacturing_PMI"
})
df_man["Date"] = pd.to_datetime(df_man["Date"], format="%Y-%m")
df_man = (
    df_man.set_index("Date")[["ISM_Manufacturing_PMI"]]
    .sort_index()
    .loc[START:END]
)

df_svc = ak.macro_usa_ism_non_pmi()

df_svc = df_svc.rename(columns={
    "日期": "Date",
    "今值": "ISM_Services_PMI"
})
df_svc["Date"] = pd.to_datetime(df_svc["Date"], format="%Y-%m")
df_svc = (
    df_svc.set_index("Date")[["ISM_Services_PMI"]]
    .sort_index()
    .loc[START:END]
)

df_us_ism = df_man.join(df_svc, how="outer")

df_us_ism.to_csv('csv_files/idea_csv/df_us_ism.csv')


In [48]:
df_us_ism = pd.read_csv('csv_files/idea_csv/df_us_ism.csv')
df_us_ism

,Date,ISM_Manufacturing_PMI,ISM_Services_PMI
0,2009-01-02,32.4,NaN
1,2009-01-06,NaN,40.6
2,2009-02-02,35.6,NaN
3,2009-02-04,NaN,42.9
4,2009-03-02,35.8,NaN
...,...,...,...
139,2014-10-03,NaN,58.6
140,2014-11-03,59.0,NaN
141,2014-11-05,NaN,57.1
142,2014-12-01,58.7,NaN


#### US CPI Food & Beverages


In [49]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_us_cpi_food = DataReader("CPIFABSL", "fred", START, END)

df_us_cpi_food.rename(columns={"CPIFABSL": "CPI_Food_Beverages"}, inplace=True)

df_us_cpi_food = (
    df_us_cpi_food["CPI_Food_Beverages"]
    .resample("W-FRI")
    .ffill()
    .to_frame()
)
df_us_cpi_food.to_csv('csv_files/idea_csv/df_us_cpi_food.csv')

In [50]:
df_us_cpi_food = pd.read_csv('csv_files/idea_csv/df_us_cpi_food.csv')
df_us_cpi_food

,DATE,CPI_Food_Beverages
0,2009-01-02,219.323
1,2009-01-09,219.323
2,2009-01-16,219.323
3,2009-01-23,219.323
4,2009-01-30,219.323
...,...,...
305,2014-11-07,245.106
306,2014-11-14,245.106
307,2014-11-21,245.106
308,2014-11-28,245.106


#### US CPI Shelter (Housing)

In [51]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_us_cpi_shelter = DataReader("CUSR0000SAH1", "fred", START, END)

df_us_cpi_shelter.rename(columns={"CUSR0000SAH1": "CPI_Shelter"}, inplace=True)

df_us_cpi_shelter = (
    df_us_cpi_shelter["CPI_Shelter"]
      .resample("W-FRI")
      .ffill()            
      .to_frame()       
)

df_us_cpi_shelter.to_csv('csv_files/idea_csv/df_us_cpi_shelter.csv')

In [52]:
df_us_cpi_shelter = pd.read_csv('csv_files/idea_csv/df_us_cpi_shelter.csv')
df_us_cpi_shelter

,DATE,CPI_Shelter
0,2009-01-02,248.963
1,2009-01-09,248.963
2,2009-01-16,248.963
3,2009-01-23,248.963
4,2009-01-30,248.963
...,...,...
305,2014-11-07,273.436
306,2014-11-14,273.436
307,2014-11-21,273.436
308,2014-11-28,273.436


#### US CPI Medical Care


In [53]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_us_cpi_med = DataReader("CPIMEDSL", "fred", START, END)

df_us_cpi_med.rename(columns={"CPIMEDSL": "CPI_Medical_Care"}, inplace=True)

df_us_cpi_med = (
    df_us_cpi_med["CPI_Medical_Care"]
      .resample("W-FRI")   # calendar‐weeks ending Fridays
      .ffill()             # carry each month’s CPI forward until the next release
      .to_frame()
)

df_us_cpi_med.to_csv('csv_files/idea_csv/df_us_cpi_med.csv')

In [54]:
df_us_cpi_med = pd.read_csv('csv_files/idea_csv/df_us_cpi_med.csv')
df_us_cpi_med

,DATE,CPI_Medical_Care
0,2009-01-02,369.824
1,2009-01-09,369.824
2,2009-01-16,369.824
3,2009-01-23,369.824
4,2009-01-30,369.824
...,...,...
305,2014-11-07,439.408
306,2014-11-14,439.408
307,2014-11-21,439.408
308,2014-11-28,439.408


#### US CPI Transportation


In [55]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_us_cpi_trans = DataReader("CPITRNSL", "fred", START, END)

df_us_cpi_trans.rename(columns={"CPITRNSL": "CPI_Transportation"}, inplace=True)

df_us_cpi_trans = (
    df_us_cpi_trans["CPI_Transportation"]
      .resample("W-FRI")
      .ffill()
      .to_frame()
)

df_us_cpi_trans.to_csv('csv_files/idea_csv/df_us_cpi_trans.csv')

In [56]:
df_us_cpi_trans = pd.read_csv('csv_files/idea_csv/df_us_cpi_trans.csv')
df_us_cpi_trans

,DATE,CPI_Transportation
0,2009-01-02,168.403
1,2009-01-09,168.403
2,2009-01-16,168.403
3,2009-01-23,168.403
4,2009-01-30,168.403
...,...,...
305,2014-11-07,210.033
306,2014-11-14,210.033
307,2014-11-21,210.033
308,2014-11-28,210.033


#### PCE: US Healthcare Services


In [57]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_us_pce_health = DataReader("DHLCRC1Q027SBEA", "fred", START, END)

df_us_pce_health.rename(columns={"DHLCRC1Q027SBEA": "PCE_Healthcare_Services"}, inplace=True)

df_us_pce_health = (
    df_us_pce_health["PCE_Healthcare_Services"]
      .resample("W-FRI")   
      .ffill()      
      .to_frame()
)


df_us_pce_health.to_csv('csv_files/idea_csv/df_us_pce_health.csv')

In [58]:
df_us_pce_health = pd.read_csv('csv_files/idea_csv/df_us_pce_health.csv')
df_us_pce_health

,DATE,PCE_Healthcare_Services
0,2009-01-02,1600.080
1,2009-01-09,1600.080
2,2009-01-16,1600.080
3,2009-01-23,1600.080
4,2009-01-30,1600.080
...,...,...
296,2014-09-05,1967.554
297,2014-09-12,1967.554
298,2014-09-19,1967.554
299,2014-09-26,1967.554


#### US ICSA (Jobless Claims)

In [59]:
%%skip
START, END = "2009-01-01", "2014-12-31"


df_us_icsa_jobless = DataReader("ICSA", "fred", START, END)

df_us_icsa_jobless.rename(columns={"ICSA": "Weekly_Initial_Jobless_Claims"}, inplace=True)

df_us_icsa_jobless = (
    df_us_icsa_jobless["Weekly_Initial_Jobless_Claims"]
      .resample("W-FRI")  
      .ffill()           
      .to_frame()
)

df_us_icsa_jobless.to_csv('csv_files/idea_csv/df_us_icsa_jobless.csv')

In [60]:
df_us_icsa_jobless = pd.read_csv('csv_files/idea_csv/df_us_icsa_jobless.csv')
df_us_icsa_jobless

,DATE,Weekly_Initial_Jobless_Claims
0,2009-01-09,503000
1,2009-01-16,551000
2,2009-01-23,591000
3,2009-01-30,586000
4,2009-02-06,629000
...,...,...
308,2014-12-05,291000
309,2014-12-12,291000
310,2014-12-19,286000
311,2014-12-26,276000


#### US Rail , Freight & Carloads

In [61]:
%%skip
START, END = "2009-01-01", "2014-12-31"

rail = DataReader("RAILFRTCARLOADS", "fred", START, END)
rail.rename(columns={"RAILFRTCARLOADS": "Rail_Freight_Carloads"}, inplace=True)

rail_weekly = (
    rail["Rail_Freight_Carloads"]
        .resample("W-FRI")
        .ffill()
        .to_frame()
)

truck = DataReader("TRUCKD11", "fred", START, END)
truck.rename(columns={"TRUCKD11": "Truck_Tonnage_Index"}, inplace=True)

truck_weekly = (
    truck["Truck_Tonnage_Index"]
         .resample("W-FRI")
         .ffill()
         .to_frame()
)

df_us_rail_freight_carloads = rail_weekly.join(truck_weekly, how="outer")

df_us_rail_freight_carloads.to_csv('csv_files/idea_csv/df_us_rail_freight_carloads.csv')


In [62]:
df_us_rail_freight_carloads = pd.read_csv('csv_files/idea_csv/df_us_rail_freight_carloads.csv')
df_us_rail_freight_carloads

,DATE,Rail_Freight_Carloads,Truck_Tonnage_Index
0,2009-01-02,1131568,77.4
1,2009-01-09,1131568,77.4
2,2009-01-16,1131568,77.4
3,2009-01-23,1131568,77.4
4,2009-01-30,1131568,77.4
...,...,...,...
305,2014-11-07,1213830,100.0
306,2014-11-14,1213830,100.0
307,2014-11-21,1213830,100.0
308,2014-11-28,1213830,100.0


#### EU PMI

In [63]:
%%skip
df_eu = ak.macro_euro_manufacturing_pmi()

print(df_eu.columns.tolist())

In [64]:
%%skip
START, END = "2009-01-01", "2014-12-31"

df_eu_pmi = ak.macro_euro_manufacturing_pmi()

df_eu_pmi = df_eu_pmi.rename(columns={
    "日期": "Date",
    "今值":  "Euro_Manufacturing_PMI"
})
df_eu_pmi["Date"] = pd.to_datetime(df_eu_pmi["Date"], format="%Y-%m")

df_eu_pmi = (
    df_eu_pmi.set_index("Date")[["Euro_Manufacturing_PMI"]]
              .sort_index()
              .loc[START:END]
)

df_eu_pmi = (
    df_eu_pmi["Euro_Manufacturing_PMI"]
      .resample("W-FRI")
      .ffill()
      .to_frame()
)
df_eu_pmi.to_csv('csv_files/idea_csv/df_eu_pmi.csv')


In [65]:
df_eu_pmi = pd.read_csv('csv_files/idea_csv/df_eu_pmi.csv')
df_eu_pmi

,Date,Euro_Manufacturing_PMI
0,2009-01-02,33.9
1,2009-01-09,33.9
2,2009-01-16,33.9
3,2009-01-23,34.5
4,2009-01-30,34.5
...,...,...
307,2014-11-21,50.4
308,2014-11-28,50.4
309,2014-12-05,50.1
310,2014-12-12,50.1


## Data Cleaning

In [66]:
df_wm_train = df_walmart_train

### Data Cleaning - Train

#### SP 500

In [67]:
df_sp500.dtypes

Date                      object
SPX_Weekly_Mean_Close    float64
dtype: object

In [68]:
df_sp500['Date'] = pd.to_datetime(df_sp500["Date"], errors="raise")

In [69]:
df_sp500

,Date,SPX_Weekly_Mean_Close
0,2000-01-07,1420.333984
1,2000-01-14,1448.648022
2,2000-01-21,1449.492493
3,2000-01-28,1394.874023
4,2000-02-04,1412.439990
...,...,...
1324,2025-05-23,5878.699902
1325,2025-05-30,5908.487427
1326,2025-06-06,5963.355957
1327,2025-06-13,6017.832031


In [70]:
df_sp500.rename(columns={
    'SPX_Weekly_Mean_Close': 'SP500_Weekly_Mean_Close'
}, inplace=True)

In [71]:
df_merged = df_wm_train.merge(
    df_sp500,
    on="Date",
    how="left",
    validate="many_to_one"
)

In [72]:
df_merged

,Store,Dept,Date,Weekly_Sales,IsHoliday,SP500_Weekly_Mean_Close
0,1,1,2010-02-05,24924.50,False,1083.817969
1,1,1,2010-02-12,46039.49,True,1069.873999
2,1,1,2010-02-19,41595.55,False,1102.575012
3,1,1,2010-02-26,19403.54,False,1103.055981
4,1,1,2010-03-05,21827.90,False,1122.895996
...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,1443.923999
421566,45,98,2012-10-05,628.10,False,1452.712012
421567,45,98,2012-10-12,1061.02,False,1438.269995
421568,45,98,2012-10-19,760.01,False,1449.297998


In [73]:
df_wm_train = df_merged

#### Walmart Stock Price

In [74]:
df_walmart_stock.dtypes


Date                      object
WMT_Weekly_Mean_Close    float64
dtype: object

In [75]:
df_walmart_stock['Date'] = pd.to_datetime(df_walmart_stock["Date"], errors="raise")

In [76]:
df_merged = df_wm_train.merge(
    df_walmart_stock,
    on="Date",
    how="left",
    validate="many_to_one"
)

In [77]:
df_wm_train = df_merged

#### External Logistic companies Walmart

In [78]:
df_logistics

,Date,ARCB_df_logistics_Close,AIT_df_logistics_Close,CEVA_df_logistics_Close,FDX_df_logistics_Close,SAIA_df_logistics_Close,TFII.TO_df_logistics_Close,XPO_df_logistics_Close,ODFL_df_logistics_Close,UPS_df_logistics_Close,JBHT_df_logistics_Close
0,2000-01-07,10.2456,4.2252,NaN,36.1330,NaN,NaN,NaN,0.4840,33.4329,2.6094
1,2000-01-14,10.2209,4.0500,NaN,37.6513,NaN,NaN,NaN,0.4914,33.7016,2.5821
2,2000-01-21,10.6344,4.3393,NaN,35.1930,NaN,NaN,NaN,0.5243,34.1281,2.5124
3,2000-01-28,9.9543,4.2880,NaN,33.1066,NaN,NaN,NaN,0.5354,32.7017,2.4714
4,2000-02-04,9.3568,4.2649,NaN,31.9144,NaN,NaN,NaN,0.5190,29.3584,2.5079
...,...,...,...,...,...,...,...,...,...,...,...
1324,2025-05-23,63.7080,226.5780,20.1420,219.8980,272.0360,121.3875,121.308,165.8267,96.9700,141.4160
1325,2025-05-30,63.9625,227.6050,19.1925,218.8175,269.3875,119.2300,117.125,161.9264,97.4825,139.3725
1326,2025-06-06,63.4820,228.9500,19.8700,218.5800,260.4360,118.7120,115.588,160.7950,97.8220,139.3440
1327,2025-06-13,69.1700,230.5940,21.0920,224.7040,259.1800,123.1960,121.270,162.9600,100.7700,141.6980


In [79]:
df_logistics.dtypes

Date                           object
ARCB_df_logistics_Close       float64
AIT_df_logistics_Close        float64
CEVA_df_logistics_Close       float64
FDX_df_logistics_Close        float64
SAIA_df_logistics_Close       float64
TFII.TO_df_logistics_Close    float64
XPO_df_logistics_Close        float64
ODFL_df_logistics_Close       float64
UPS_df_logistics_Close        float64
JBHT_df_logistics_Close       float64
dtype: object

In [80]:
df_logistics['Date'] = pd.to_datetime(df_logistics["Date"], errors="raise")

In [81]:
df_merged = df_wm_train.merge(
    df_logistics,
    on="Date",
    how="left",
    validate="many_to_one"
)

In [82]:
df_wm_train = df_merged

#### Official China PMI (Caixin PMI only starts in 2014)

In [83]:
df_pmi_china

,Date,Official_Manufacturing_PMI,Official_Manufacturing_PMI_YoY,Official_Services_PMI,Official_Services_PMI_YoY
0,2009-01-01,45.3,-14.528302,53.7,-10.797342
1,2009-02-01,49.0,-8.239700,55.1,-7.082631
2,2009-03-01,52.4,-10.273973,54.4,-7.640068
3,2009-04-01,53.5,-9.628378,53.5,-8.390411
4,2009-05-01,53.1,-0.375235,54.9,-4.355401
...,...,...,...,...,...
67,2014-08-01,51.1,0.196078,54.4,0.927644
68,2014-09-01,51.1,0.000000,54.0,-2.527076
69,2014-10-01,50.8,-1.167315,53.8,-4.440497
70,2014-11-01,50.3,-2.140078,53.9,-3.750000


In [84]:
df_wm_train

,Store,Dept,Date,Weekly_Sales,IsHoliday,SP500_Weekly_Mean_Close,WMT_Weekly_Mean_Close,ARCB_df_logistics_Close,AIT_df_logistics_Close,CEVA_df_logistics_Close,FDX_df_logistics_Close,SAIA_df_logistics_Close,TFII.TO_df_logistics_Close,XPO_df_logistics_Close,ODFL_df_logistics_Close,UPS_df_logistics_Close,JBHT_df_logistics_Close
0,1,1,2010-02-05,24924.50,False,1083.817969,12.849577,19.7749,16.3716,11.386,67.5286,8.1133,5.9522,1.7320,4.0012,35.1546,26.3251
1,1,1,2010-02-12,46039.49,True,1069.873999,12.741080,19.9706,16.2364,11.006,66.1162,8.1400,5.7845,1.7154,4.0306,34.4932,26.7885
2,1,1,2010-02-19,41595.55,False,1102.575012,12.876699,21.8988,16.8577,11.700,67.7903,7.6600,5.8391,1.8538,4.1189,35.0687,27.9924
3,1,1,2010-02-26,19403.54,False,1103.055981,12.942228,22.6758,17.1335,11.864,69.9236,8.0707,6.2608,1.9285,4.3097,35.6784,29.4901
4,1,1,2010-03-05,21827.90,False,1122.895996,12.925909,23.9654,17.7752,12.288,72.5866,8.4240,6.8441,1.9368,4.3818,36.1191,29.9464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False,1443.923999,19.073621,7.1706,33.7323,14.684,72.7305,13.5920,13.5184,4.2125,9.6705,47.1874,46.1906
421566,45,98,2012-10-05,628.10,False,1452.712012,19.118869,6.9492,33.4393,13.892,73.3577,13.8053,13.6795,4.2215,9.4961,47.7151,47.3806
421567,45,98,2012-10-12,1061.02,False,1438.269995,19.313219,7.3217,32.9701,13.610,75.7907,13.8760,13.5060,4.1440,9.4916,47.7099,49.2570
421568,45,98,2012-10-19,760.01,False,1449.297998,19.706032,7.2911,32.7124,14.746,79.0085,14.3053,13.6934,4.3412,9.8127,47.9002,51.6494


In [85]:
df_pmi_china.drop(columns=['Official_Manufacturing_PMI_YoY', 'Official_Services_PMI_YoY'], inplace=True)

In [86]:
df_pmi_china.rename(columns={
    'Official_Manufacturing_PMI': 'China_Official_Manufacturing_PMI',
    'Official_Services_PMI': 'China_Official_Services_PMI'
}, inplace=True)

In [87]:
df_pmi_china['Date'] = pd.to_datetime(df_pmi_china["Date"], errors="raise")

In [88]:
df_wm_train['YM'] = df_wm_train['Date'].dt.to_period('M')
df_pmi_china    ['YM'] = df_pmi_china    ['Date'].dt.to_period('M')

df_merged = df_wm_train.merge(
    df_pmi_china[['YM','China_Official_Manufacturing_PMI','China_Official_Services_PMI']],
    on='YM',
    how='left'
).drop(columns='YM')


In [89]:
df_wm_train = df_merged

#### PCE USA (Personal Consumption Expenditures)

In [90]:
df_pce

,DATE,Personal_Consumption_Expenditures
0,2000-01-01,6542.9
1,2000-02-01,6625.3
2,2000-03-01,6686.5
3,2000-04-01,6679.1
4,2000-05-01,6709.7
...,...,...
299,2024-12-01,20408.1
300,2025-01-01,20389.0
301,2025-02-01,20469.3
302,2025-03-01,20621.8


In [91]:
df_pce.rename(columns={
    'Personal_Consumption_Expenditures': 'US_Personal_Consumption_Expenditures'
}, inplace=True)

In [92]:
df_pce['DATE'] = pd.to_datetime(df_pce["DATE"], errors="raise")

In [93]:
df_wm_train['YM'] = df_wm_train['Date'].dt.to_period('M')
df_pce    ['YM'] = df_pce    ['DATE'].dt.to_period('M')

df_merged = df_wm_train.merge(
    df_pce[['YM','US_Personal_Consumption_Expenditures']],
    on='YM',
    how='left'
).drop(columns='YM')

In [94]:
df_wm_train = df_merged

#### Interest Rates USA (Fed Funds Rate & Tbill 3 Months Yield)

In [95]:
print(df_interest_rates)

            DATE  Fed_Funds_Rate  TBill_3mo_Yield
0     2001-06-01            3.97            3.670
1     2001-06-08             NaN            3.640
2     2001-06-15             NaN            3.558
3     2001-06-22             NaN            3.496
4     2001-06-29             NaN            3.546
...          ...             ...              ...
1250  2025-05-16             NaN            4.398
1251  2025-05-23             NaN            4.372
1252  2025-05-30             NaN            4.355
1253  2025-06-06             NaN            4.436
1254  2025-06-13             NaN            4.450

[1255 rows x 3 columns]


In [96]:
df_interest_rates['DATE'] = pd.to_datetime(df_interest_rates["DATE"], errors="raise")

In [97]:
df_interest_rates.rename(columns={
    'Fed_Funds_Rate': 'US_Fed_Funds_Rate',
    'TBill_3mo_Yield': 'US_TBill_3mo_Yield',
    'DATE': 'Date'

}, inplace=True)

In [98]:
df_wm_train = df_wm_train.sort_values('Date')
df_interest_rates = df_interest_rates.sort_values('Date')

df_merged = df_wm_train.merge(
    df_interest_rates[['Date','US_TBill_3mo_Yield','US_Fed_Funds_Rate']],
    on='Date',
    how='left'
)

df_merged['US_Fed_Funds_Rate'] = df_merged['US_Fed_Funds_Rate'].ffill()

In [99]:
df_wm_train = df_merged

#### CCI USA (Consumer Confidence Index) from University of Michigan

In [100]:
df_us_cci


,DATE,Consumer_Sentiment_UMich
0,2001-07-01,92.4
1,2001-08-01,91.5
2,2001-09-01,81.8
3,2001-10-01,82.7
4,2001-11-01,83.9
...,...,...
281,2024-12-01,74.0
282,2025-01-01,71.7
283,2025-02-01,64.7
284,2025-03-01,57.0


In [101]:
df_us_cci['Date']   = pd.to_datetime(df_us_cci['DATE'])
df_us_cci = df_us_cci.sort_values('Date')

df_merged = pd.merge_asof(
    df_wm_train.sort_values('Date'),
    df_us_cci[['Date','Consumer_Sentiment_UMich']],
    on='Date',
    direction='backward'
)

In [102]:
df_wm_train = df_merged

#### U.S.A Advance Retail Sales: Retail Trade and Food Services

In [103]:
df_us_retail

,DATE,Retail_Sales_Retail_and_Food_Services_USA
0,2000-01-01,261545
1,2000-02-01,265686
2,2000-03-01,269019
3,2000-04-01,264067
4,2000-05-01,265992
...,...,...
300,2025-01-01,711461
301,2025-02-01,711757
302,2025-03-01,722572
303,2025-04-01,721983


In [104]:
df_us_retail['Date']   = pd.to_datetime(df_us_retail['DATE'])
df_us_retail = df_us_retail.sort_values('Date')

df_merged = pd.merge_asof(
    df_wm_train.sort_values('Date'),
    df_us_retail[['Date','Retail_Sales_Retail_and_Food_Services_USA']],
    on='Date',
    direction='backward'
)

In [105]:
df_wm_train = df_merged

#### Exchange Rates (China, Mexico, Canada, India, Vietnam)

In [106]:
df_fx

,Unnamed: 0,CNY_per_USD,MXN_per_USD,CAD_per_USD,INR_per_USD,"('VND_per_USD', 'USDVND=X')"
0,2001-06-22,8.2771,9.0712,1.5308,47.028,NaN
1,2001-06-29,8.2768,9.0695,1.5184,47.064,NaN
2,2001-07-06,8.2768,9.0672,1.5146,47.175,NaN
3,2001-07-13,8.2768,9.2520,1.5270,47.200,NaN
4,2001-07-20,8.2768,9.1962,1.5404,47.162,NaN
...,...,...,...,...,...,...
1248,2025-05-23,7.2038,19.3119,1.3864,85.558,25944.4
1249,2025-05-30,7.1934,19.3442,1.3788,85.420,25938.4
1250,2025-06-06,7.1865,19.2013,1.3688,85.646,26041.2
1251,2025-06-13,7.1827,18.9487,1.3642,85.670,26018.4


In [107]:
df_fx.rename(columns={
    'Unnamed: 0': 'Date',
    "('VND_per_USD', 'USDVND=X')": 'VND_per_USD'
}, inplace=True)

In [108]:
df_fx['Date']   = pd.to_datetime(df_fx['Date'])
df_fx = df_fx.sort_values('Date')

df_merged = pd.merge_asof(
    df_wm_train.sort_values('Date'),
    df_fx[['Date','CNY_per_USD', 'MXN_per_USD','CAD_per_USD','INR_per_USD','VND_per_USD']],
    on='Date',
    direction='backward'
)

In [109]:
df_wm_train = df_merged

#### US External Tax Rate

In [110]:
df_us_tariff

,Year,Canada,China,India,Mexico,Viet Nam
0,2000,1.31,14.67,23.36,15.15,NaN
1,2001,1.60,14.11,26.51,15.24,15.29
2,2002,1.49,7.72,23.08,5.03,14.73
3,2003,2.46,6.48,21.45,15.44,11.79
4,2004,1.51,5.96,22.96,3.00,11.69
5,2005,1.46,4.87,13.90,3.00,11.35
6,2006,1.45,4.25,8.99,2.39,10.21
7,2007,1.54,5.07,11.99,2.41,9.13
8,2008,1.54,4.47,5.98,1.87,5.03
9,2009,1.70,3.94,6.84,1.61,6.76


In [111]:
df_us_tariff.rename(columns={
    'Canada': 'US_TAX_Canada',
    'China': 'US_TAX_China',
    'India': 'US_TAX_India',
    'Mexico': 'US_TAX_Mexico',
    'Viet Nam': 'US_TAX_Vietnam'
}, inplace=True)

In [112]:
import pandas as pd

# list of tariff columns
tariff_cols = [
    'US_TAX_Canada',
    'US_TAX_China',
    'US_TAX_India',
    'US_TAX_Mexico',
    'US_TAX_Vietnam'
]

# 1) prepare the dates
df_wm_train['Date']      = pd.to_datetime(df_wm_train['Date'])
df_us_tariff['Date']     = pd.date_range(
    '2001-01-01',
    periods=len(df_us_tariff),
    freq='YS'
)

# 2) sort & merge_asof, then forward-fill tariffs
df_merged = (
    pd.merge_asof(
        df_wm_train.sort_values('Date'),
        df_us_tariff[['Date'] + tariff_cols].sort_values('Date'),
        on='Date',
        direction='backward'
    )
    .assign(**{col: lambda d, col=col: d[col].ffill() for col in tariff_cols})
)


In [113]:
df_unique = df_merged.drop_duplicates(subset='Date', keep='first')

In [114]:
df_wm_train = df_merged

#### Tax Return (Train)

In [115]:
df_tax_return_train

,Date,TaxReturnImpact
0,2010-02-05,0.0
1,2010-02-12,0.0
2,2010-02-19,0.0
3,2010-02-26,0.0
4,2010-03-05,0.0
...,...,...
421565,2012-09-28,0.0
421566,2012-10-05,0.0
421567,2012-10-12,0.0
421568,2012-10-19,0.0


In [116]:
df_tax_return_train.rename(columns={
    'TaxReturnImpact': 'US_Tax_Return'
}, inplace=True)

In [117]:
df_tax_return_train['Date']   = pd.to_datetime(df_tax_return_train['Date'])
df_tax_return_train = df_tax_return_train.sort_values('Date')

df_merged = pd.merge_asof(
    df_wm_train.sort_values('Date'),
    df_tax_return_train[['Date','US_Tax_Return']],
    on='Date',
    direction='backward'
)

In [118]:
df_unique = df_merged.drop_duplicates(subset='Date', keep='first')

In [119]:
df_wm_train = df_merged

In [120]:
df_wm_train

,Store,Dept,Date,Weekly_Sales,IsHoliday,SP500_Weekly_Mean_Close,WMT_Weekly_Mean_Close,ARCB_df_logistics_Close,AIT_df_logistics_Close,CEVA_df_logistics_Close,...,MXN_per_USD,CAD_per_USD,INR_per_USD,VND_per_USD,US_TAX_Canada,US_TAX_China,US_TAX_India,US_TAX_Mexico,US_TAX_Vietnam,US_Tax_Return
0,1,1,2010-02-05,24924.50,False,1083.817969,12.849577,19.7749,16.3716,11.386,...,13.0110,1.0666,46.256,18421.0,1.70,3.94,6.84,1.61,6.76,0.0
1,9,97,2010-02-05,668.48,False,1083.817969,12.849577,19.7749,16.3716,11.386,...,13.0110,1.0666,46.256,18421.0,1.70,3.94,6.84,1.61,6.76,0.0
2,9,85,2010-02-05,693.87,False,1083.817969,12.849577,19.7749,16.3716,11.386,...,13.0110,1.0666,46.256,18421.0,1.70,3.94,6.84,1.61,6.76,0.0
3,8,80,2010-02-05,8654.60,False,1083.817969,12.849577,19.7749,16.3716,11.386,...,13.0110,1.0666,46.256,18421.0,1.70,3.94,6.84,1.61,6.76,0.0
4,9,55,2010-02-05,11123.56,False,1083.817969,12.849577,19.7749,16.3716,11.386,...,13.0110,1.0666,46.256,18421.0,1.70,3.94,6.84,1.61,6.76,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,36,42,2012-10-26,149.64,False,1416.117969,19.314762,6.9204,31.1912,14.826,...,12.9683,0.9950,53.612,20652.0,1.38,5.99,7.33,5.37,5.02,0.0
421566,27,3,2012-10-26,12431.35,False,1416.117969,19.314762,6.9204,31.1912,14.826,...,12.9683,0.9950,53.612,20652.0,1.38,5.99,7.33,5.37,5.02,0.0
421567,2,59,2012-10-26,287.25,False,1416.117969,19.314762,6.9204,31.1912,14.826,...,12.9683,0.9950,53.612,20652.0,1.38,5.99,7.33,5.37,5.02,0.0
421568,32,67,2012-10-26,8464.65,False,1416.117969,19.314762,6.9204,31.1912,14.826,...,12.9683,0.9950,53.612,20652.0,1.38,5.99,7.33,5.37,5.02,0.0


## Data Transformation